# Working with Message Stack payloads 
- there are currently three support schemes; openai, google, anthropic
- these have slightyl different payload structure for tool calls and responses in particular
- generally, we want to ACK a tool call with an id and follow it with a response in the message stack
- anthropic has a tool block and google has a functionResponse while open AI is easier with just the typically message with role and content
- we can test read message stacks as instructions from the database in different contexts
    - for a user question; trivial 
    - for a tool request with tool stack
    - for agents that provider system prompts

In [8]:
import sys
sys.path.append('../')
import percolate as p8
from percolate.models.p8 import AIResponse
from percolate.services import PostgresService
from percolate.services.llm.LanguageModel import request_anthropic,request_google,request_openai
from pydantic import BaseModel, model_validator, Field
import typing
import uuid

pg = PostgresService()

### illustrate that in each scheme we read messages that are ready for that scheme and functions too
- there is a corresponding database request_x that reads data in the same way

## The first test should take a singel turn example id
- at this point you have created a request with any scheme using e.g. percolate_with_agent


In [11]:
test_session_id = '8c51c161-7ac8-db55-68ec-7255ea4983e0'

In [12]:
goo_mm =  [d for d in pg.execute(f""" select * from p8.get_google_messages('{test_session_id}') """)[0]['messages']]  
fns =  [d for d in pg.execute(f""" select * from p8.get_tools_by_name(ARRAY['get_pet_findByStatus'],'google') """)[0]['get_tools_by_name']]  
request_google(goo_mm,fns).json()

In [13]:
ant_mm = [d for d in pg.execute(f""" select * from p8.get_anthropic_messages('{test_session_id}') """)[0]['messages']]
fns =  [d for d in pg.execute(f""" select * from p8.get_tools_by_name(ARRAY['get_pet_findByStatus'],'anthropic') """)[0]['get_tools_by_name']]  
request_anthropic(ant_mm,fns).json()

In [14]:
mm = [d for d in pg.execute(f""" select * from p8.get_canonical_messages('{test_session_id}') """)[0]['messages']]
fns =  [d for d in pg.execute(f""" select * from p8.get_tools_by_name(ARRAY['get_pet_findByStatus']) """)[0]['get_tools_by_name']]  
request_openai(mm,fns).json()

## Longer turn tests
- make sure function calls and responses are paired properly 
- test injecting in new quuestions